## SFMFB Fundraiser - WEGE 2021 - Rankings

Last Updated: 12:04PM (PST) 12/10/21

In [25]:
import pandas as pd
pd.set_option('display.max_colwidth', 80)
import numpy as np
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, TableColumn, DataTable
from bokeh.io import show
# !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML

In [2]:
HTML('''<script>
code_show = true;
function code_toggle(){
    if(code_show){
        $('div.input').hide();
    }else{
        $('div.input').show();
    }
    code_show = !code_show
}
$(document).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off raw code"></form>''')

In [20]:
## Successful Leaderboard Creation on all???
# starttime = time.time()
print("Loading all pages... sit tight - this will take a minute!")

driver = webdriver.Chrome()
url = 'https://us-p2p.e-activist.com/1803/general/donate'
driver.get(url)
time.sleep(3)
prev_hgt = driver.execute_script('return document.body.scrollHeight')
element = driver.find_element_by_tag_name('body')
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.3)
    new_hgt = driver.execute_script('return document.body.scrollHeight')
    if(new_hgt == prev_hgt):
        break;
    else:
        prev_hgt=new_hgt

print("Finished loading page")
fundraisers = driver.find_elements_by_class_name("donate-result-block ")
print("\n","Total # Fundraising Events Found: ", len(fundraisers))

print(" Generating Live Leaderboard...")
titleArray = np.array([]);
grpArray = np.array([]);
amtArray = np.array([]);
for fundraiser in fundraisers:
    title = fundraiser.find_element_by_class_name('donate-result-page').text
    amtRaised = fundraiser.find_element_by_class_name('donate-result-raised').text

    try:
        grpName = fundraiser.find_element_by_class_name('donate-result-group').text
    except:
        try:
            grpName = fundraiser.find_element_by_class_name('donate-result-desc').text
        except:
            grpName = ''
    
    titleArray = np.append(titleArray, title)
    grpArray = np.append(grpArray, grpName)
    amtArray = np.append(amtArray, amtRaised)
    
df = pd.DataFrame({"Fundraising Event": titleArray, "Group Name":grpArray, "Raised":amtArray})
# df.tail(50)
driver.quit()

leader = df[(df["Raised"]!='$0 RAISED') | (df["Fundraising Event"] == "FICO Scores B2B Holiday Fundraiser - WEGE 2021")]
leader["Raised"].replace(to_replace=",", value="", regex=True, inplace=True)
leader["Raised"].replace(to_replace="\$([0-9,\.]+).*", value=r"\1", regex=True, inplace=True)
leader["Raised"] = leader["Raised"].astype("double")
leader.sort_values(by='Raised', ascending=False, inplace = True)
leader.reset_index(inplace = True)
leader.insert(0, 'Rank', np.arange(1, len(leader)+1))
leader = leader.drop("index", axis=1)

print("\n","Total # Active Fundraisers Found: ", len(leader))


idx = pd.IndexSlice
slice_ = idx[idx[leader['Fundraising Event'] =="FICO Scores B2B Holiday Fundraiser - WEGE 2021"], :]

def style_bold(val):
    return "font-weight: bold"

scoresRank = leader[leader['Fundraising Event'] =="FICO Scores B2B Holiday Fundraiser - WEGE 2021"].Rank.iloc[0]
scores = leader[(leader['Rank'] >= scoresRank-5) & ((leader['Rank'] <= scoresRank+5) )]
scores.style.set_properties(**{'background-color': 'gold'}, subset=slice_).format(formatter={('Raised'): "{:.2f}"})\
    .applymap(style_bold, subset=slice_)

# endtime = time.time()
# print("Run Time :", endtime-starttime)


Loading all pages... sit tight - this will take a minute!
Finished loading page

 Total # Fundraising Events Found:  426
 Generating Live Leaderboard...

 Total # Active Fundraisers Found:  299


,Rank,Fundraising Event,Group Name,Raised
293,294,San Francisco Airport Marriott Waterfront - Holiday Fundraiser,San Francisco Airport Marriott Waterfront (Group Fundraiser),25.00
294,295,Adrienne Gembala,Adrienne Gembala,25.00
295,296,Colette's Fundraising Drive for SF-Marin Food Bank,Colette Whitney,12.24
296,297,Rachel's Fundraiser,Rachel Breuer,5.00
297,298,Firefly's Fantastic Virtual Food Drive,Firefly Partners (Group Fundraiser),2.04
298,299,FICO Scores B2B Holiday Fundraiser - WEGE 2021,FICO Scores B2B Fundraising Page,0.00


In [21]:
leader.style.set_properties(**{'background-color': 'gold'}, subset=slice_).format(formatter={('Raised'): "{:.2f}"})\
    .applymap(style_bold, subset=slice_)

,Rank,Fundraising Event,Group Name,Raised
0,1,Incite.org Holiday Support for SF-Marin Food Bank,Incite.org Holiday Support for SF-Marin Food Bank,254727.50
1,2,Lick-Wilmerding High School Fund Drive,Lick-Wilmerding High School,16608.90
2,3,Lucasfilm Trivia Night 2021,Trivia Night 2021,13308.72
3,4,Day One Biopharmaceuticals Inc.,"Day One Biopharmaceuticals, Inc.",12812.00
4,5,St. Cecilia School Joins the University of Notre Dame's Meals with Muffet in Hunger Action Month,Michelle McCauley,12218.56
5,6,Abstract Thanksgiving Fundraiser 2020,Abstract Thanksgiving Fundraiser,10019.00
6,7,Feeding Families,Paradise Cay Financial,10013.00
7,8,Feeding Friendzy,Feeding Friendzy,10000.00
8,9,Volley 2021 Giving Page,Volley (Group Fundraiser),8374.22
9,10,County of Marin Holiday Food Drive,County of Marin,7275.00
